In [9]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import csv, random, time

import contractions
from bs4 import BeautifulSoup
import unicodedata
import re
import torch.nn as nn
import torch

from utils import *


In [4]:
dataset = []
with open('data/snips_processed/snips.csv', 'r') as f:
    reader = csv.reader(x.replace('\0', '') for x in f)
    for line in reader:
        dataset.append(line)
dataset = np.array(dataset)

testset = dataset[10000 :]

trainset = dataset[:10000]

train_sent_en = trainset[:,1]
#train_sent_sv = trainset[:,2]
train_lab = trainset[:,0]

test_sent_en = testset[:,1]
#test_sent_sv = testset[:,2]
test_lab = testset[:,0]

labels = ['BookRestaurant','GetWeather', 'SearchScreeningEvent','RateBook', 'SearchCreativeWork', 'AddToPlaylist', 'PlayMusic']
lab2id = {}
id2lab = {}

for i in range(len(labels)):
    lab2id[labels[i]] = i
    id2lab[i] = labels[i]

In [109]:
len(trainset), len(testset), trainset[4]

(10000, 3784, array(['RateBook',
        'I give The Lady Decides a rating value of 4 and a best rating of 6',
        'Jag ger Damen Bestämmer sig för ett betyg värde av 4 och en bäst betyg av 6'],
       dtype='<U187'))

In [111]:
import requests
def get_vecs(sents):
    
    r = requests.post(
            'http://localhost:8501/v1/models/universal_encoder:predict', 
            json={'inputs': {'text': sents}})

    responce = r.json() 
    return responce['outputs']

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


In [5]:
def prepare_labs(labs):
    out = []
    for lab in labs:
        out.append(lab2id[lab])
    return out

In [114]:
labs = prepare_labs(dataset[:,0])
test_labs = prepare_labs(test_lab)

train_sent_en = list(pre_process_documents(train_sent_en))

train_vecs = []
for i in range(0,len(train_sent_en),100):
    train_vecs.extend(get_vecs(train_sent_en[i:i+100]))

    
test_sent_en = list(pre_process_documents(test_sent_en))

test_vecs = []
for i in range(0,len(test_sent_en),100):
    test_vecs.extend(get_vecs(test_sent_en[i:i+100]))
    
test_vecs = list(map(lambda x: np.array([x]) , test_vecs))
train_vecs = list(map(lambda x: np.array([x]) , train_vecs))

In [8]:
vecs_en = np.load('data/snips_processed/USE-en.npy')


In [49]:
def full_train_model(train_vecs, train_labs, test_vecs, test_labs, verbose = False, runs = 6001):
    net = Baseline(in_size=512)
    optimizer = torch.optim.Adam(net.parameters())
    criterion = torch.nn.NLLLoss()
    
    tvecs = torch.tensor(list(train_vecs)).float()
    tvecst = torch.tensor(list(test_vecs)).float()
    tlabs = torch.tensor(train_labs).long()
    tlabst = torch.tensor(test_labs).long()
    
    t = time.time()
    for i in range(runs):
        loss = train(net, criterion, optimizer, tlabs, tvecs)
    
        if verbose and not i% 100:
            eval_loss, acc = evaluate(net, tlabst, tvecst, criterion)
            print('#{:3d}, {:5d} sec. train loss: {:.7f}, eval loss: {:.4f}, acc = {:.3f}'.format(i, int(time.time() - t), loss, eval_loss, acc))
    
    eval_loss, acc = evaluate(net, tlabst, tvecst, criterion)
    return acc, net
    

In [50]:
def cv(vecs, labs, folds = 5):
    
    delims = np.arange(0, len(vecs), len(vecs)//folds)
    results = []
    t = time.time()
    for i in range(folds):
        acc, model = full_train_model(np.vstack((vecs[:delims[i]], vecs[delims[i+1]:])),
                             np.hstack((labs[:delims[i]], labs[delims[i+1]:])),
                             vecs[delims[i] : delims[i+1]],
                             labs[delims[i] : delims[i+1]],
                                  False, runs = 6001)
        
        results.append(acc)
        print('#{:3d}, {:5d} sec. acc = {:.3f}'.format(i, int(time.time() - t), results[-1]))

    return(sum(results)/len(results))   

In [51]:
cv(vecs_en, labs)

#  0,    82 sec. acc = 0.972
#  1,   171 sec. acc = 0.968
#  2,   254 sec. acc = 0.969
#  3,   338 sec. acc = 0.967
#  4,   421 sec. acc = 0.968


0.9687953555878084

In [15]:
acc, net = full_train_model(vecs_en[:10000], labs[:10000], vecs_en[10000:], labs[10000:], verbose=True)

#  0,     0 sec. train loss: 0.0001947, eval loss: 1.9383, acc = 0.197
#100,     1 sec. train loss: 0.0001265, eval loss: 1.2599, acc = 0.915
#200,     2 sec. train loss: 0.0000860, eval loss: 0.8596, acc = 0.921
#300,     3 sec. train loss: 0.0000635, eval loss: 0.6377, acc = 0.928
#400,     4 sec. train loss: 0.0000503, eval loss: 0.5076, acc = 0.934
#500,     6 sec. train loss: 0.0000419, eval loss: 0.4247, acc = 0.938
#600,     7 sec. train loss: 0.0000361, eval loss: 0.3680, acc = 0.943
#700,     8 sec. train loss: 0.0000319, eval loss: 0.3270, acc = 0.946
#800,     9 sec. train loss: 0.0000287, eval loss: 0.2960, acc = 0.950
#900,    11 sec. train loss: 0.0000263, eval loss: 0.2717, acc = 0.952
#1000,    12 sec. train loss: 0.0000243, eval loss: 0.2523, acc = 0.953
#1100,    13 sec. train loss: 0.0000226, eval loss: 0.2363, acc = 0.954
#1200,    14 sec. train loss: 0.0000212, eval loss: 0.2229, acc = 0.957
#1300,    15 sec. train loss: 0.0000201, eval loss: 0.2115, acc = 0.958
#1

In [127]:
vecs = []

a = list(pre_process_documents(dataset[:,1]))
print(a[1:10])
for i in range(0,len(a),100):
    vecs.extend(get_vecs(a[i:i+100]))
    print(i//100)
    
vecs = list(map(lambda x: np.array([x]) , vecs))

['give this album of points', 'book a table at atlantic grill in lofgreen', 'give the current album out of points', 'i give the lady decides a rating value of and a best rating of', 'add la voce to my dubstep dangles dirty playlist', 'play any chanson', 'rate the previous essay four of points', 'give the current novel a one out of rating', 'this current book should get four stars or a rating of']
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137


In [99]:
nptv2s = np.array(train_vecs)

In [128]:
np.save('data/snips_processed/USE-en',np.array(vecs))

In [7]:
labs = prepare_labs(dataset[:, 0])
np.save('data/snips_processed/labs',np.array(labs))

In [16]:
torch.save(net, 'models/USE-linear')


In [17]:
vecs_en

array([[[-0.0796285 ,  0.012136  , -0.0135948 , ...,  0.0257535 ,
         -0.0545727 , -0.0220814 ]],

       [[-0.0503377 , -0.0464248 , -0.00572828, ...,  0.0480531 ,
         -0.0832616 , -0.078685  ]],

       [[ 0.0322778 ,  0.0720855 , -0.0252778 , ..., -0.00387515,
         -0.0430343 ,  0.0628465 ]],

       ...,

       [[-0.0255634 ,  0.00681807, -0.0536128 , ...,  0.0411098 ,
          0.0321836 , -0.0290044 ]],

       [[-0.0690401 ,  0.0629174 , -0.0154625 , ...,  0.0387612 ,
         -0.0643083 , -0.0636527 ]],

       [[ 0.0332168 , -0.0266076 , -0.00943216, ...,  0.0420742 ,
         -0.0401178 , -0.0227483 ]]])